In [119]:
# code to polynomial interpolation using divided difference
import numpy as np
import cmath
# x and y values
def divided_diff(x, y):
    n = len(x)
    F_matrix = np.zeros((n, n))
    F_matrix = np.diag(y)
    for j in range(0,n):
        i=j-1
        while i>=0:
            # print("xi-xj", x[i], x[j])
            F_matrix[i,j]=(F_matrix[i,j-1]-F_matrix[i+1,j])/(x[i]-x[j])
            i=i-1
    return F_matrix

def get_quadratic_coeff(F, x):
    # print(F, x)
    a = F[0,2]
    c = F[0,0] + x[0]*x[1]*F[0,2] -x[0]*F[0,1]
    b = F[0,1] -(x[0]+x[1])*F[0,2]
    return a,b,c

def get_quadratic_roots(a, b, c):
    D = b**2 - 4*a*c
    if D >= 0:
        x1 = (-b + np.sqrt(D)) / (2*a)
        x2 = (-b - np.sqrt(D)) / (2*a)
    else:
        x1 = (-b + cmath.sqrt(D)) / (2*a)
        x2 = (-b - cmath.sqrt(D)) / (2*a)
    return x1, x2

In [158]:
def poly(x, roots=[]):
    if roots == []:
        return x**3 + x + 1
    else:
        p = 1
        for r in roots:
            p = p*(x-r)
        return (x**3 + x + 1)/p

In [164]:
def mueller_method_1root(x, y, roots):
    for i in range(1000):
        F = divided_diff(x, y)
        a,b,c = get_quadratic_coeff(F, x)
        # print(a,b,c)
        r1, r2 = get_quadratic_roots(a,b,c)
        if abs(poly(r1, roots)) < 1e-5:
            return r1
        if abs(poly(r2, roots)) < 1e-5:
            return r2
        else:
            if abs(poly(r1, roots)) < abs(poly(r2, roots)):
                x = [x[1], x[2], r1]
                y = [y[1], y[2], poly(r1, roots)]
            else:
                x = [x[1], x[2], r2]
                y = [y[1], y[2], poly(r2, roots)]
    return None

# x is random 3 numbers in range -10 to 10
x = [0, 1, 2]
y = [poly(i) for i in x]


r1 = mueller_method_1root(x, y, [])
print(r1, poly(r1))

# x is random 3 numbers in range -10 to 10
x = [0, 1, -5]
y = [poly(i, [r1]) for i in x]

r2 = mueller_method_1root(x, y, [r1])
print(r2, poly(r2))

# x is random 3 numbers in range -10 to 10
x = [0, 1, -5]
y = [poly(i, [r1, r2]) for i in x]

r3 = mueller_method_1root(x, y, [r1, r2])
print(r3, poly(r3))

(0.3411638900222406-1.1615413814668338j) (7.614719410575788e-08-2.1727144527972087e-08j)
(0.34116389673131825+1.1615413785979467j) (6.486493564494822e-08+4.5420303296239695e-08j)
(-0.682327826114347-0j) (-5.341394770930208e-08+0j)


In [118]:
print(r1*r2*r3)

(-0.9999987204233751+1.879084529643471e-07j)


In [4]:
import cmath
import numpy as np

def muellers_method(f, x0, x1, x2, tol=1e-6, max_iter=100):
    """
    Implements Mueller's method for finding roots of a function.
    
    Parameters:
    f (function): The function for which we are trying to find a root.
    x0, x1, x2 (float): Initial guesses for the root.
    tol (float): Tolerance for convergence.
    max_iter (int): Maximum number of iterations.
    
    Returns:
    float: The root of the function.
    """
    for _ in range(max_iter):
        h1 = x1 - x0
        h2 = x2 - x1
        delta1 = (f(x1) - f(x0)) / h1
        delta2 = (f(x2) - f(x1)) / h2
        a = (delta2 - delta1) / (h2 + h1)
        b = a * h2 + delta2
        c = f(x2)
        
        rad = cmath.sqrt(b**2 - 4*a*c)
        if abs(b + rad) > abs(b - rad):
            den = b + rad
        else:
            den = b - rad
        
        dxr = -2 * c / den
        x3 = x2 + dxr
        
        if abs(dxr) < tol:
            return x3
        
        x0, x1, x2 = x1, x2, x3
    
    raise ValueError("Maximum number of iterations reached without convergence")

def deflate_polynomial(coeffs, root):
    """
    Deflates the polynomial by dividing it by (x - root).
    
    Parameters:
    coeffs (list): Coefficients of the polynomial.
    root (complex): The root to deflate by.
    
    Returns:
    list: Coefficients of the deflated polynomial.
    """
    new_coeffs = []
    new_coeffs.append(coeffs[0])
    for i in range(1, len(coeffs)):
        new_coeffs.append(coeffs[i] + new_coeffs[i-1] * root)
    return new_coeffs[:-1]

# Define the polynomial coefficients (highest degree first)
coeffs = [1, 0, 1, 1]

# Find all roots
roots = []
for _ in range(len(coeffs) - 1):
    # Define the polynomial function
    def f(x):
        return sum(c * x**i for i, c in enumerate(reversed(coeffs)))
    
    # Initial guesses
    x0, x1, x2 = 0.5, 1.5, 2.5
    
    # Find the root
    root = muellers_method(f, x0, x1, x2)
    roots.append(root)
    
    # Deflate the polynomial
    coeffs = deflate_polynomial(coeffs, root)

print(f"The roots are: {roots}")

The roots are: [(0.34116390191400964-1.1615413999972517j), (0.3411639019140094+1.161541399997252j), (-0.682327803828019-2.220446049250313e-16j)]
